In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, UpSampling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import l2

def conv_block(input, filters):

    skip = Conv2D(filters, 1, padding="same")(input)
    x = BatchNormalization()(input)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Add()([x, skip])
    x = Dropout(0.2)(x)
    return x

def decoder_block(x, skip, filters):
    x = Conv2DTranspose(filters, (3, 3), strides=2, padding="same")(x)
    x = Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x


def feature_pyramid(feature_maps):
    d1,d2,d3,d4,d5 = feature_maps

    p1 = conv_block(d1, 42)
    p2 = Add()([UpSampling2D(size=(2, 2))(p1), Conv2D(42, 1, padding="same")(d2)])
    p3 = Add()([UpSampling2D(size=(2, 2))(p2), Conv2D(42, 1, padding="same")(d3)])
    p4 = Add()([UpSampling2D(size=(2, 2))(p3), Conv2D(42, 1, padding="same")(d4)])
    p5 = Add()([UpSampling2D(size=(2, 2))(p4), Conv2D(42, 1, padding="same")(d5)])

    return p5

def build_fcn_with_fpn(output_channels: int):
    # Load EfficientNetB0 model as backbone
    backbone = effnet
    layer1 = backbone.get_layer('input_1').output #224 3
    layer1 = conv_block(layer1, 64) #double number of channels
    layer1 = Dropout(0.2)(layer1)  # Adding dropout with a rate of 0.2 as specified in the paper

    layer3 = backbone.get_layer('block1a_activation').output #112 96
    layer4 = backbone.get_layer('block2a_activation').output #56 144
    layer5 = backbone.get_layer('block3a_activation').output #14x14x240
    layer6 = backbone.get_layer('block4a_activation').output
    layer7 = backbone.get_layer('top_activation').output

    #upsampling decoder
    d1 = decoder_block(layer7, layer6, 288) #
    d2 = decoder_block(d1, layer5, 144) #
    d3 = decoder_block(d2, layer4, 72) #
    d4 = decoder_block(d3, layer3, 52) #
    d5 = decoder_block(d4, layer1, 32) #
    #21 classes
    outputs = feature_pyramid([d1,d2,d3,d4,d5])
    outputs = Conv2D(21, 1, padding="same", activation="softmax")(outputs) #use softmax for multiclass detection instead of binary
    # Construct the FCN model with FPN
    model = Model(inputs=backbone.inputs, outputs=outputs)

    return model

# Build the model
fcn_fpn_model = build_fcn_with_fpn(output_channels=21)

# Print model summary
fcn_fpn_model.summary()